# Importation des bibliothèques nécessaires

In [ ]:
# Importation de la bibliothèque BeautifulSoup, renommée ici en 'bs' pour faciliter son utilisation
from bs4 import BeautifulSoup as bs

# Importation de la bibliothèque 'os' pour interagir avec le système de fichiers et gérer les opérations liées aux chemins
import os

# Importation de la bibliothèque 'Workbook' pour créer et manipuler des fichiers Excel
from openpyxl import Workbook

# Importation de la bibliothèque Selenium pour automatiser le navigateur Web
from selenium import webdriver

# Importation de la classe 'By' de Selenium pour identifier les éléments HTML à travers différents sélecteurs (id, classe, nom, etc.)
from selenium.webdriver.common.by import By

# Importation de la classe 'WebDriverWait' de Selenium pour implémenter une attente explicite jusqu'à ce qu'une condition soit remplie
from selenium.webdriver.support.ui import WebDriverWait

# Importation de la classe 'expected_conditions' pour spécifier les conditions d'attente (ex: présence d'un élément sur la page)
from selenium.webdriver.support import expected_conditions as EC 

# Importation de 'TimeoutException' pour gérer les exceptions liées aux dépassements de temps lors de l'attente d'un élément
from selenium.common.exceptions import TimeoutException

# Importation de la bibliothèque 'time' pour gérer les pauses et temporisations dans l'exécution du script
import time

# Importation de la bibliothèque 'requests' pour effectuer des requêtes HTTP (GET, POST, etc.)
import requests


# Création de répertoire

Ce segment de code est utilisé pour initialiser deux répertoires dans le système de fichiers : `PDFs` et `Fichiers_Excel`.
Le premier est destiné à stocker les fichiers PDF, et le second pour les fichiers Excel générés ou utilisés par le script.
Cette étape garantit que les dossiers sont créés au début de l'exécution du programme, afin d'éviter toute erreur lors de la tentative de sauvegarde ou de manipulation de fichiers.

**Objectif** :
- Créer des répertoires spécifiques pour organiser et gérer les fichiers PDF et Excel générés par le script.
- Prévenir les erreurs liées à l'absence de dossiers.

**Prérequis** :
- Avoir les permissions nécessaires pour créer des dossiers dans le répertoire courant.
- Le module `os` est inclus dans Python par défaut, donc aucune installation supplémentaire n'est requise.

**Explications détaillées** :
- `os.makedirs(folder, exist_ok=True)` :
  - **`folder`** : Spécifie le nom ou le chemin du répertoire à créer.
  - **`exist_ok=True`** : Permet de ne pas générer d'erreur si le répertoire existe déjà. Cela empêche le programme de planter en cas de réexécution.

**Avantages** :
- **Robustesse** : Assure que les dossiers nécessaires existent avant tout traitement.
- **Gestion facilitée** : Centralise le stockage des fichiers pour une récupération et une organisation plus simples.

In [ ]:

# Utilisé pour stocker tous les fichiers PDF générés ou récupérés pendant l'exécution du programme
pdf_folder = 'PDFs'

# Création du dossier 'PDFs' s'il n'existe pas déjà.
# 'exist_ok=True' permet d'éviter une erreur si le dossier est déjà présent dans le répertoire courant.
os.makedirs(pdf_folder, exist_ok=True)


# Ce dossier est utilisé pour stocker les fichiers Excel créés ou modifiés par le script
excel_folder = 'Fichiers_Excel'

# Création du dossier 'Fichiers_Excel' s'il n'existe pas déjà.
# 'exist_ok=True' a le même effet qu'auparavant : il permet d'éviter une exception si le dossier existe déjà.
os.makedirs(excel_folder, exist_ok=True)


# Refus de cookies

Cette fonction `click_refuse_youtube_cookies(driver)` est conçue pour automatiser le processus de refus des cookies sur YouTube à l'aide de Selenium.
Lorsqu'une fenêtre contextuelle de consentement apparaît, elle clique sur le bouton 'Tout refuser' pour éviter le stockage de cookies non désirés. 

**Objectif** :
- Éviter l'enregistrement des cookies en refusant systématiquement toutes les options proposées par YouTube.

**Contexte** :
- Utilisé principalement dans des scripts de web scraping ou d'automatisation où l'on souhaite naviguer sur YouTube sans accepter les cookies.
- Cette fonction est appelée chaque fois qu'une fenêtre de consentement apparaît, ce qui est typique lors de la première connexion à YouTube.

**Approche** :
1. Mettre en pause le script pendant 2 secondes pour laisser le temps à la page de se charger.
2. Localiser le bouton 'Tout refuser' en utilisant son chemin XPath et attendre qu'il soit cliquable.
3. Exécuter un clic JavaScript pour contourner les éventuels masques de superposition.
4. Attendre que le bouton soit cliqué correctement.
5. Gérer les exceptions en cas de problèmes (élément non trouvé, timeout, etc.).

**Avantages** :
- Assure que les cookies ne seront pas enregistrés sans le consentement de l'utilisateur.
- Permet de continuer l'automatisation sans être interrompu par les fenêtres de consentement.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le pilote correspondant au navigateur utilisé (`chromedriver` pour Chrome, par exemple) doit être configuré.
- Cette fonction doit être appelée avec un objet `driver` déjà initialisé pour YouTube.


In [ ]:

# Définition de la fonction pour refuser les cookies sur YouTube
def click_refuse_youtube_cookies(driver):
    try:
        # Mettre en pause le script pendant 2 secondes pour laisser le temps à la page de charger le bouton
        time.sleep(2)

        # Localiser le bouton 'Tout refuser' avec un XPath et attendre qu'il soit cliquable
        refuse_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button//span[text()='Tout refuser']"))
        )

        # Exécuter un clic JavaScript pour éviter les masques de superposition qui pourraient empêcher le clic normal
        driver.execute_script("arguments[0].click();", refuse_button)

        # Attendre jusqu'à ce qu'un autre élément (lié au refus de consentement) soit cliquable pour confirmer que le clic a réussi
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.XPATH, '/html/body/ytd-app/ytd-consent-bump-v2-lightbox/tp-yt-paper-dialog/div[4]/div[2]/div[6]/div[1]/ytd-button-renderer[1]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
        ))

        # Cliquer sur le bouton 'Tout refuser' après avoir vérifié qu'il est bien présent et actif
        refuse_button.click()

        # Afficher un message de succès dans la console
        print("Bouton 'Tout refuser' cliqué sur YouTube.")

    except Exception as e:
        # Afficher un message d'erreur si le refus des cookies échoue, ainsi que le message de l'exception
        print(f"Erreur lors du refus des cookies sur YouTube : {e}")


# Recherhche par thème

La fonction `search_by_theme(driver, theme)` effectue une recherche sur un site web en fonction d'un thème ou mot-clé fourni en paramètre. 
Elle utilise Selenium pour interagir avec les champs de recherche et les boutons du site. Cette fonction est utile pour automatiser 
des recherches basées sur des mots-clés thématiques, que ce soit pour l'extraction de données, la navigation automatisée ou pour tester des fonctionnalités.

**Objectif** :
- Automatiser la saisie et la recherche de mots-clés dans un champ de recherche d'un site web.

**Contexte** :
- Typiquement utilisée dans le cadre de l'automatisation des tests ou du scraping de contenu basé sur des thèmes spécifiques.
- Permet d'interagir avec un champ de recherche et de lancer la recherche pour naviguer vers la page de résultats.

**Approche** :
1. Utiliser `WebDriverWait` pour attendre que le champ de recherche soit visible.
2. Saisir le thème dans le champ de recherche après l'avoir vidé pour s'assurer qu'aucun texte résiduel n'interfère.
3. Localiser le bouton de recherche et attendre qu'il soit cliquable.
4. Lancer la recherche en cliquant sur le bouton.
5. Afficher un message de succès ou gérer les erreurs si l'un des éléments n'est pas trouvé.

**Avantages** :
- Évite les erreurs d'interaction (par exemple, cliquer trop tôt sur le bouton).
- Assure que les interactions sont synchronisées avec le chargement de la page.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- La page sur laquelle le driver navigue doit contenir un champ de recherche avec le nom `mot_cle`.
- Le driver doit être initialisé et pointé vers la bonne page avant l'appel de cette fonction.

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.
- `theme` : Thème ou mot-clé à rechercher (chaîne de caractères).

**Exceptions gérées** :
- `TimeoutException` : Levée si le champ de recherche ou le bouton n'apparaissent pas dans le délai imparti.
- `NoSuchElementException` : Levée si l'un des éléments n'est pas présent sur la page.

In [ ]:

# Définition de la fonction pour effectuer une recherche par thème sur un site web donné
def search_by_theme(driver, theme):
    try:
        # Attendre que le champ de recherche soit présent sur la page avec un délai maximum de 20 secondes
        search_input = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.NAME, "mot_cle"))  # Identification du champ de recherche par son nom
        )

        # Effacer le contenu existant du champ de recherche pour éviter tout conflit avec le nouveau texte
        search_input.clear()

        # Saisir le mot-clé (thème) dans le champ de recherche
        search_input.send_keys(theme)

        # Attendre que le bouton de recherche soit cliquable avec un délai de 20 secondes
        search_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "input[title='Lancer la recherche'][type='submit']"))  # Sélection par attribut CSS
        )

        # Cliquer sur le bouton pour lancer la recherche
        search_button.click()

        # Afficher un message dans la console pour indiquer le succès de l'opération
        print(f"Recherche lancée pour le thème : {theme}")

    except Exception as e:
        # Afficher un message d'erreur en cas de problème, avec les détails de l'exception capturée
        print(f"Erreur lors de la recherche : {e}")


# Vérification des résulats

La fonction `check_no_results(driver)` permet de vérifier si aucun résultat n'est trouvé après avoir effectué un défilement sur une page de résultats. 
Elle utilise Selenium pour contrôler le défilement et inspecter le contenu de la page à la recherche d'un message ou de l'absence d'éléments indiquant qu'aucun résultat n'est disponible.

**Objectif** :
- Détecter et signaler l'absence de résultats après un défilement sur une page web.

**Contexte** :
- Utilisé lors de l'exploration de pages web comportant des résultats paginés ou chargés dynamiquement (scroll infini).
- Permet de s'arrêter automatiquement lors d'un scraping lorsqu'aucun résultat n'est trouvé, optimisant ainsi les ressources et le temps d'exécution.

**Approche** :
1. Effectuer un seul défilement de la page pour charger les nouveaux contenus.
2. Vérifier la présence d'un message indiquant qu'il n'y a pas de résultats.
3. Si le message n'est pas présent, vérifier s'il n'y a aucun élément correspondant aux résultats attendus sur la page.
4. Retourner `True` si aucun résultat n'est trouvé, sinon `False`.

**Avantages** :
- Simplifie la gestion des pages sans résultats, évitant ainsi des traitements inutiles.
- Améliore la robustesse du script en gérant les pages sans contenu ou partiellement chargées.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le driver doit être initialisé et pointé vers une page de résultats avant l'appel de cette fonction.

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.

**Exceptions gérées** :
- `NoSuchElementException` : Levée si les éléments recherchés ne sont pas présents sur la page.
- `TimeoutException` : Levée si le chargement ou l'exécution du JavaScript prend trop de temps.

**Retour** :
- `True` : Si aucun résultat n'est trouvé après le défilement.
- `False` : Si des résultats sont présents.

In [ ]:


# Définition de la fonction pour vérifier la présence ou l'absence de résultats après un défilement
def check_no_results(driver):
    # Définir le texte indicatif de l'absence de résultats sur la page
    no_results_text = "Nous n'avons pas de ressources disponibles avec ces critères"

    try:
        # Effectuer un défilement vers le bas pour charger plus de contenu (scroll vers le bas d'une hauteur de fenêtre)
        driver.execute_script("window.scrollBy(0, window.innerHeight);")

        # Attendre 2 secondes pour laisser le temps au contenu de se charger après le défilement
        time.sleep(2)

        # Rechercher un élément de type div qui contient un message indiquant qu'il n'y a pas de résultats
        header = driver.find_elements(By.CSS_SELECTOR, "div.views-empty")

        # Vérifier si le message d'absence de résultats est présent dans l'élément trouvé
        if header and no_results_text in header[0].text:
            # Si le message est trouvé, afficher un message et retourner True
            print("Aucun résultat trouvé pour ce thème.")
            return True

        # Rechercher les éléments correspondant aux résultats (ex: éléments de type div avec la classe 'views-row')
        results = driver.find_elements(By.CSS_SELECTOR, ".views-row")

        # Si la liste de résultats est vide, cela signifie qu'aucun résultat n'est présent après le défilement
        if not results:
            # Afficher un message dans la console et retourner True
            print("Aucun résultat trouvé après un scroll.")
            return True

        # Si des résultats sont trouvés, retourner False
        return False

    except Exception as e:
        # En cas d'erreur, afficher un message d'erreur dans la console avec le détail de l'exception
        print(f"Erreur lors de la vérification des résultats : {str(e)}")
        # Retourner False pour indiquer que la vérification a échoué
        return False


# Téléchargement de pdf

La fonction `download_pdf(url, filename)` permet de télécharger un fichier PDF depuis une URL spécifiée, de le sauvegarder localement avec le nom de fichier indiqué, 
et de gérer les éventuelles erreurs rencontrées lors du téléchargement. Cette fonction vérifie d'abord la validité de l'URL, puis effectue une requête pour récupérer le contenu du PDF, 
et enfin l'enregistre dans le dossier `PDFs`.

**Objectif** :
- Télécharger un fichier PDF à partir d'une URL et le sauvegarder localement avec un nom de fichier spécifique.

**Contexte** :
- Utile dans des scripts d'extraction de données (scraping) ou d'automatisation pour récupérer des documents PDF de manière systématique.
- Peut être utilisé dans des contextes où les documents doivent être archivés pour une consultation hors ligne ou pour des traitements ultérieurs.

**Approche** :
1. Vérifier et ajuster l'URL pour s'assurer qu'elle commence par `http://` ou `https://`.
2. Effectuer une requête HTTP pour récupérer le contenu du PDF.
3. Vérifier le succès de la requête (code 200).
4. Enregistrer le fichier localement dans le dossier `PDFs` en mode binaire (`wb`).
5. Gérer les exceptions spécifiques aux requêtes (problème de réseau, URL invalide) et les autres exceptions.

**Avantages** :
- Permet de récupérer des fichiers PDF depuis diverses sources en s'assurant que le fichier est correctement téléchargé et sauvegardé.
- Gestion d'erreurs intégrée pour éviter les crashs du programme en cas d'échec de téléchargement.

**Prérequis** :
- Le module `requests` doit être installé (`pip install requests`).
- Le module `os` doit être importé pour gérer les chemins de fichiers.
- Le dossier `PDFs` doit exister ou avoir été créé auparavant dans le script.

**Arguments** :
- `url` : URL du fichier PDF à télécharger.
- `filename` : Nom sous lequel le fichier doit être sauvegardé localement (incluant l'extension `.pdf`).

**Exceptions gérées** :
- `requests.exceptions.RequestException` : Levée en cas de problème de connexion, URL invalide ou autre erreur de requête.
- `Exception` : Gère toute autre erreur inattendue (par exemple, problème d'écriture de fichier).

**Retour** :
- Aucun retour spécifique, mais affiche le statut du téléchargement (succès ou échec) dans la console.


In [ ]:


# Définition de la fonction pour télécharger un fichier PDF depuis une URL spécifiée
def download_pdf(url, filename):
    try:
        # Vérifier et ajuster l'URL si nécessaire (ajoute 'https://' si manquant)
        if not (url.startswith('http://') or url.startswith('https://')):
            # Ajouter 'https://' au début de l'URL si elle commence par des slashes ou est incomplète
            url = 'https://' + url.lstrip('/')

        # Envoyer une requête GET à l'URL pour récupérer le fichier PDF
        response = requests.get(url)

        # Lever une exception si la requête échoue (mauvaise réponse HTTP)
        response.raise_for_status()

        # Vérifier si la réponse est réussie (code 200)
        if response.status_code == 200:
            # Construire le chemin complet du fichier PDF à sauvegarder localement
            full_path = os.path.join(pdf_folder, filename)

            # Ouvrir le fichier en mode binaire (écriture) et y sauvegarder le contenu téléchargé
            with open(full_path, 'wb') as f:
                f.write(response.content)

            # Afficher un message de succès si le fichier est téléchargé avec succès
            print(f"Téléchargement PDF réussi : {full_path}")
        else:
            # Afficher un message d'erreur si la réponse HTTP n'est pas correcte (ex: code 404)
            print(f"Échec du téléchargement PDF : {filename}")

    # Gérer les erreurs de requête spécifiques (problème de connexion, URL invalide, etc.)
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement du PDF {filename} : {str(e)}")

    # Gérer toute autre erreur inattendue (ex: problème d'écriture dans le fichier)
    except Exception as e:
        print(f"Une erreur inattendue est survenue : {str(e)}")


# Export des données dans un fichier excel

La fonction `export_to_excel(data)` permet de créer et d'enregistrer un fichier Excel à partir de données fournies sous forme de liste de listes.
Elle organise les informations dans un format structuré en ajoutant des en-têtes de colonnes et les lignes de données correspondantes.
Cette fonction utilise le module `openpyxl` pour manipuler les fichiers Excel.

**Objectif** :
- Exporter des données structurées dans un fichier Excel pour faciliter l'analyse et la consultation.

**Contexte** :
- Cette fonction est utilisée lorsqu'il est nécessaire de sauvegarder des informations collectées dans un format tabulaire (Excel), 
  par exemple après l'extraction de données d'un site ou d'une base de données.
- Elle permet de centraliser les informations dans un document unique et facilement lisible.

**Approche** :
1. Créer un nouveau classeur Excel (`Workbook`).
2. Ajouter une feuille intitulée "Informations générales" pour organiser les données.
3. Définir les en-têtes de colonnes correspondant aux champs de données à exporter.
4. Ajouter chaque ligne de données à la feuille Excel.
5. Sauvegarder le classeur dans le dossier `Fichiers_Excel` avec un nom de fichier spécifique.
6. Gérer les erreurs liées à la création ou à la sauvegarde du fichier.

**Avantages** :
- Permet d'avoir une sauvegarde des informations sous un format standard et universellement lisible.
- Facilite le partage et l'analyse des données collectées.

**Prérequis** :
- Le module `openpyxl` doit être installé (`pip install openpyxl`).
- Le module `os` est utilisé pour gérer les chemins de fichiers.
- Le dossier `Fichiers_Excel` doit exister ou avoir été créé au préalable dans le script.

**Arguments** :
- `data` : Une liste de listes, où chaque liste interne représente une ligne à ajouter au fichier Excel.

**Exceptions gérées** :
- `Exception` : Capture toute erreur générale (problème d'écriture, de structure de données, ou de permissions).

**Retour** :
- Aucun retour spécifique, mais affiche le statut de l'exportation (succès ou échec) dans la console.

In [ ]:


# Définition de la fonction pour exporter les données vers un fichier Excel
def export_to_excel(data):
    try:
        # Créer un nouveau classeur Excel
        wb = Workbook()

        # Créer la première feuille de calcul et la nommer "Informations générales"
        ws_general = wb.active
        ws_general.title = 'Informations générales'

        # Définir les en-têtes de colonnes correspondant aux champs de données à exporter
        headers = [
            'Titre', 'Thématique', 'Description', 'Type_document', 'Source', 'Lieu', 'Lien', 'Propriétaire_video', 
            'Titre_vidéo', 'Nombre de commentaires de la vidéo', 'Nombre de vues de la vidéo', 'Nombre de likes de la vidéo',
            'Description de la vidéo', 'Lien de la vidéo', 'Date de publication'
        ]

        # Ajouter les en-têtes comme première ligne de la feuille de calcul
        ws_general.append(headers)

        # Ajouter chaque ligne de données fournie dans la liste `data` à la feuille Excel
        for line in data:
            # Chaque élément de `line` représente une ligne à ajouter au tableau
            ws_general.append(line)

        # Définir le chemin complet du fichier Excel dans le dossier `Fichiers_Excel`
        excel_filename = os.path.join(excel_folder, 'data_Dispositif_Rex.xlsx')

        # Sauvegarder le classeur Excel sous le nom 'data_Dispositif_Rex.xlsx' dans le dossier
        wb.save(excel_filename)

        # Afficher un message dans la console indiquant le succès de l'exportation
        print(f"Données exportées avec succès vers : {excel_filename}")

    except Exception as e:
        # En cas de problème lors de la création ou de la sauvegarde du fichier Excel, afficher un message d'erreur
        print(f"Une erreur est survenue lors de l'exportation vers Excel : {str(e)}")


# Récupération du nombre de commentaire

La fonction `get_nombre_commentaire(driver, video_url)` permet de récupérer le nombre de commentaires d'une vidéo YouTube en utilisant Selenium pour naviguer sur la page. 
Elle prend en compte les cas où les commentaires sont désactivés et gère les erreurs liées à l'extraction des informations.

**Objectif** :
- Accéder à une vidéo YouTube et extraire le nombre de commentaires affichés.

**Contexte** :
- Utilisée dans des scripts de collecte de données pour analyser l'engagement des utilisateurs sur des vidéos YouTube.
- Pratique pour récupérer des métriques supplémentaires telles que le nombre de commentaires, vues ou likes, pour des analyses comparatives.

**Approche** :
1. Charger la page de la vidéo en utilisant `driver.get(video_url)`.
2. Scroller vers le bas de la page pour forcer le chargement des commentaires.
3. Vérifier si les commentaires sont désactivés en recherchant un message spécifique.
4. Si les commentaires sont activés, récupérer le nombre total de commentaires.
5. Gérer les exceptions pour garantir que la fonction ne plante pas en cas de problème.

**Avantages** :
- Gère automatiquement les cas où les commentaires sont désactivés.
- Utilise des techniques de défilement et d'attente pour s'assurer que tous les éléments sont chargés avant de tenter de les lire.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le pilote de navigateur (ex. `chromedriver`) doit être configuré correctement.
- La page de la vidéo doit être accessible (pas de restrictions d'âge, géographiques, ou de permissions).

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.
- `video_url` : URL de la vidéo YouTube pour laquelle le nombre de commentaires doit être extrait.

**Exceptions gérées** :
- `TimeoutException` : Levée si l'élément (nombre de commentaires ou message de désactivation) n'est pas trouvé dans le délai imparti.
- `Exception` : Gère toute autre erreur liée au chargement de la page ou à l'extraction des informations.

**Retour** :
- `nombre_commentaires` : Chaîne de caractères représentant le nombre de commentaires (ou "N/A" si les commentaires ne sont pas disponibles).


In [ ]:

# Définition de la fonction pour extraire le nombre de commentaires d'une vidéo YouTube
def get_nombre_commentaire(driver, video_url):
    try:
        # Charger la page de la vidéo spécifiée par `video_url`
        driver.get(video_url)

        # Scroller vers le bas de la page pour charger la section des commentaires
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Attendre quelques secondes pour s'assurer que la page a bien défilé et que les commentaires sont chargés
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))  # Vérifier que le corps de la page est bien chargé
        )

        # Essayer de détecter si un message indique que les commentaires sont désactivés
        try:
            commentaire_desactive_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//yt-formatted-string[contains(text(), 'Les commentaires sont désactivés')]"))
            )

            # Si l'élément indiquant que les commentaires sont désactivés est trouvé, retourner "N/A"
            if commentaire_desactive_element:
                nombre_commentaires = "N/A"
                print("Les commentaires sont désactivés sur cette vidéo.")

        # Gérer l'exception si le message de désactivation n'est pas trouvé (donc les commentaires ne sont pas désactivés)
        except TimeoutException:
            try:
                # Extraire le nombre de commentaires si l'élément correspondant est trouvé
                nombre_commentaires = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'yt-formatted-string.count-text.style-scope.ytd-comments-header-renderer'))
                ).text
            except Exception as e:
                # Si une autre erreur survient pendant l'extraction du nombre de commentaires, définir `nombre_commentaires` comme "N/A"
                nombre_commentaires = "N/A"
                print(f"Erreur lors de l'extraction du nombre de commentaires : {str(e)}")

    # Gérer les erreurs générales (par exemple, problème lors du chargement de la vidéo)
    except Exception as e:
        nombre_commentaires = "N/A"
        print(f"Erreur lors du chargement de la vidéo : {str(e)}")

    # Retourner le nombre de commentaires récupéré ou "N/A" si une erreur s'est produite
    return nombre_commentaires


# Récupérer le titre de la vidéo

La fonction `get_titre_video(driver, video_url)` permet d'extraire le titre d'une vidéo YouTube en utilisant Selenium pour naviguer sur la page de la vidéo.
Elle vérifie que la page est bien chargée, puis localise l'élément contenant le titre de la vidéo et retourne ce texte. 

**Objectif** :
- Accéder à une vidéo YouTube et extraire son titre.

**Contexte** :
- Utilisé principalement dans des scripts de collecte de données YouTube pour récupérer des informations sur les vidéos, telles que le titre, les vues, ou les commentaires.
- Pratique pour construire des bases de données de vidéos ou pour analyser le contenu.

**Approche** :
1. Charger la page de la vidéo en utilisant `driver.get(video_url)`.
2. Attendre que l'élément contenant le titre de la vidéo soit visible sur la page.
3. Extraire le texte de cet élément.
4. Retourner le titre ou "N/A" si l'élément n'est pas trouvé ou qu'une erreur survient.

**Avantages** :
- La fonction gère les cas où le titre n'est pas disponible (par exemple, si la page n'est pas chargée correctement).
- Utilise des techniques de `WebDriverWait` pour s'assurer que l'élément est bien présent avant de tenter de le lire.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le pilote de navigateur (ex. `chromedriver`) doit être configuré correctement.
- La page de la vidéo doit être accessible (pas de restrictions d'âge, géographiques, ou de permissions).

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.
- `video_url` : URL de la vidéo YouTube pour laquelle le titre doit être extrait.

**Exceptions gérées** :
- `TimeoutException` : Levée si l'élément (titre de la vidéo) n'est pas trouvé dans le délai imparti.
- `NoSuchElementException` : Levée si l'élément est introuvable sur la page.
- `Exception` : Capture toute autre erreur liée au chargement de la page ou à l'extraction du titre.

**Retour** :
- `titre_video` : Chaîne de caractères représentant le titre de la vidéo, ou "N/A" si le titre n'est pas disponible.


In [ ]:

# Définition de la fonction pour extraire le titre d'une vidéo YouTube
def get_titre_video(driver, video_url):
    try:
        # Charger la page de la vidéo spécifiée par `video_url`
        driver.get(video_url)

        # Attendre que l'élément contenant le titre de la vidéo soit visible sur la page
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.style-scope.ytd-watch-metadata yt-formatted-string.style-scope.ytd-watch-metadata'))
        )

        # Localiser l'élément contenant le titre de la vidéo
        titre_video_element = driver.find_element(By.CSS_SELECTOR, 'div.style-scope.ytd-watch-metadata yt-formatted-string.style-scope.ytd-watch-metadata')

        # Retourner le texte du titre s'il est trouvé, sinon "N/A"
        return titre_video_element.text.strip() if titre_video_element else 'N/A'

    except Exception as e:
        # En cas d'erreur, afficher un message et retourner "N/A"
        print(f"Erreur lors de la récupération du titre de la vidéo : {str(e)}")
        return 'N/A'


# Récupérer le nom de la chaîne

La fonction `get_video_owner(driver, video_url)` permet de récupérer le nom du propriétaire (auteur) d'une vidéo YouTube. 
Elle utilise Selenium pour naviguer sur la page de la vidéo et localiser l'élément HTML contenant le nom du propriétaire.

**Objectif** :
- Accéder à la page d'une vidéo YouTube et extraire le nom de la chaîne ou de l'utilisateur qui a publié la vidéo.

**Contexte** :
- Utile pour la collecte de données sur les vidéos, notamment pour obtenir des informations sur les auteurs et analyser la répartition des vidéos par chaîne.
- Utilisé dans des scripts de scraping ou d'automatisation pour regrouper des informations sur plusieurs vidéos d'une même chaîne.

**Approche** :
1. Charger la page de la vidéo avec `driver.get(video_url)`.
2. Attendre que l'élément contenant le nom du propriétaire de la vidéo soit visible.
3. Localiser l'élément spécifique avec un sélecteur CSS (`By.CSS_SELECTOR`).
4. Récupérer le texte correspondant au nom du propriétaire.
5. Retourner le nom ou "N/A" si l'élément n'est pas trouvé ou qu'une erreur survient.

**Avantages** :
- La fonction utilise des techniques de `WebDriverWait` pour s'assurer que l'élément est bien présent avant de tenter de le lire.
- Permet de gérer les erreurs liées au chargement de la page ou à des changements dans la structure HTML de YouTube.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le pilote de navigateur (ex. `chromedriver`) doit être configuré correctement.
- La page de la vidéo doit être accessible (pas de restrictions d'âge, géographiques, ou de permissions).

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.
- `video_url` : URL de la vidéo YouTube pour laquelle le nom du propriétaire doit être extrait.

**Exceptions gérées** :
- `TimeoutException` : Levée si l'élément contenant le nom du propriétaire n'est pas trouvé dans le délai imparti.
- `NoSuchElementException` : Levée si l'élément est introuvable sur la page.
- `Exception` : Capture toute autre erreur liée au chargement de la page ou à l'extraction du nom.

**Retour** :
- `owner_name` : Chaîne de caractères représentant le nom de l'auteur de la vidéo, ou "N/A" si le nom n'est pas disponible.


In [ ]:
# Définition de la fonction pour extraire le nom du propriétaire de la vidéo YouTube
def get_video_owner(driver, video_url):
    try:
        # Charger la page de la vidéo spécifiée par `video_url`
        driver.get(video_url)

        # Attendre que l'élément contenant le nom du propriétaire de la vidéo soit visible sur la page
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.style-scope ytd-channel-name a.yt-simple-endpoint.style-scope.yt-formatted-string'))
        )

        # Localiser l'élément contenant le nom du propriétaire de la chaîne YouTube
        owner_element = driver.find_element(By.CSS_SELECTOR, 'div.style-scope ytd-channel-name a.yt-simple-endpoint.style-scope.yt-formatted-string')

        # Retourner le texte du nom du propriétaire s'il est trouvé, sinon "N/A"
        return owner_element.text.strip() if owner_element else 'N/A'

    except Exception as e:
        # En cas d'erreur, afficher un message et retourner "N/A"
        print(f"Erreur lors de la récupération du propriétaire de la vidéo : {str(e)}")
        return 'N/A'


# Récupération du nombre de vues

La fonction `get_nombre_vues(driver, video_url)` permet de récupérer le nombre de vues d'une vidéo YouTube en utilisant Selenium pour naviguer sur la page de la vidéo.
Elle vérifie la présence de l'élément contenant le nombre de vues, l'extrait, et le retourne sous forme de texte.

**Objectif** :
- Accéder à la page d'une vidéo YouTube et extraire le nombre de vues affiché.

**Contexte** :
- Utilisé dans des scripts de collecte de données sur YouTube pour obtenir des informations sur les performances des vidéos (nombre de vues).
- Pratique pour analyser l'engagement des utilisateurs sur une vidéo ou pour comparer la popularité de plusieurs vidéos.

**Approche** :
1. Charger la page de la vidéo en utilisant `driver.get(video_url)`.
2. Attendre que l'élément contenant le nombre de vues soit visible sur la page.
3. Localiser cet élément avec un sélecteur CSS (`By.CSS_SELECTOR`).
4. Extraire le texte représentant le nombre de vues.
5. Retourner le nombre de vues ou "N/A" si l'élément n'est pas trouvé ou qu'une erreur survient.

**Avantages** :
- La fonction utilise des techniques de `WebDriverWait` pour s'assurer que l'élément est bien présent avant de tenter de le lire.
- Gère les erreurs et retourne "N/A" si le nombre de vues n'est pas disponible.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le pilote de navigateur (ex. `chromedriver`) doit être configuré correctement.
- La page de la vidéo doit être accessible (pas de restrictions d'âge, géographiques, ou de permissions).

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.
- `video_url` : URL de la vidéo YouTube pour laquelle le nombre de vues doit être extrait.

**Exceptions gérées** :
- `TimeoutException` : Levée si l'élément contenant le nombre de vues n'est pas trouvé dans le délai imparti.
- `NoSuchElementException` : Levée si l'élément est introuvable sur la page.
- `Exception` : Capture toute autre erreur liée au chargement de la page ou à l'extraction des vues.

**Retour** :
- `nombre_vues` : Chaîne de caractères représentant le nombre de vues de la vidéo, ou "N/A" si le nombre n'est pas disponible.


In [ ]:

# Définition de la fonction pour extraire le nombre de vues d'une vidéo YouTube
def get_nombre_vues(driver, video_url):
    try:
        # Charger la page de la vidéo spécifiée par `video_url`
        driver.get(video_url)

        # Attendre que l'élément contenant le nombre de vues soit visible sur la page
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'yt-formatted-string.style-scope.ytd-watch-info-text span.style-scope.yt-formatted-string.bold'))
        )

        # Localiser l'élément contenant le nombre de vues de la vidéo
        vues_element = driver.find_element(By.CSS_SELECTOR, 'yt-formatted-string.style-scope.ytd-watch-info-text span.style-scope.yt-formatted-string.bold')

        # Retourner le texte du nombre de vues s'il est trouvé, sinon "N/A"
        return vues_element.text.strip() if vues_element else 'N/A'

    except Exception as e:
        # En cas d'erreur, afficher un message et retourner "N/A"
        print(f"Erreur lors de la récupération du nombre de vues : {str(e)}")
        return 'N/A'


# Récupération du nombre de likes

La fonction `get_nombre_likes(driver, video_url)` permet de récupérer le nombre de "likes" d'une vidéo YouTube en utilisant Selenium pour naviguer sur la page de la vidéo.
Elle vérifie la présence de l'élément contenant le nombre de likes, l'extrait, et le retourne sous forme de texte. Si le texte de l'élément est "J'aime" (sans nombre), 
cela signifie que le nombre de likes est nul, et la fonction retourne `0`.

**Objectif** :
- Accéder à la page d'une vidéo YouTube et extraire le nombre de "likes" affiché sous le bouton "J'aime".

**Contexte** :
- Utile pour la collecte de données sur les vidéos YouTube afin de mesurer l'engagement du public à travers les interactions.
- Pratique pour l'analyse de la popularité d'une vidéo ou pour comparer plusieurs vidéos en fonction de leur nombre de likes.

**Approche** :
1. Charger la page de la vidéo en utilisant `driver.get(video_url)`.
2. Attendre que l'élément contenant le nombre de likes soit visible sur la page.
3. Localiser cet élément avec un sélecteur XPath (`By.XPATH`).
4. Extraire le texte représentant le nombre de likes.
5. Retourner `0` si le texte est "J'aime" (signifiant qu'il n'y a pas de likes) ou retourner le nombre extrait.
6. Retourner "N/A" si l'élément n'est pas trouvé ou qu'une erreur survient.

**Avantages** :
- La fonction utilise des techniques de `WebDriverWait` pour s'assurer que l'élément est bien présent avant de tenter de le lire.
- Gère les erreurs et retourne "N/A" si le nombre de likes n'est pas disponible.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le pilote de navigateur (ex. `chromedriver`) doit être configuré correctement.
- La page de la vidéo doit être accessible (pas de restrictions d'âge, géographiques, ou de permissions).

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.
- `video_url` : URL de la vidéo YouTube pour laquelle le nombre de likes doit être extrait.

**Exceptions gérées** :
- `TimeoutException` : Levée si l'élément contenant le nombre de likes n'est pas trouvé dans le délai imparti.
- `NoSuchElementException` : Levée si l'élément est introuvable sur la page.
- `Exception` : Capture toute autre erreur liée au chargement de la page ou à l'extraction des likes.

**Retour** :
- `nombre_likes` : Nombre de "likes" sous forme de chaîne de caractères, ou `0` si aucun like n'est enregistré, ou "N/A" si le nombre n'est pas disponible.

In [ ]:


# Définition de la fonction pour extraire le nombre de likes d'une vidéo YouTube
def get_nombre_likes(driver, video_url):
    try:
        # Accéder à l'URL de la vidéo spécifiée par `video_url`
        driver.get(video_url)

        # Attendre que l'élément contenant le nombre de likes soit présent sur la page
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[2]/div[2]/div/div/ytd-menu-renderer/div[1]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]'))
        )

        # Localiser l'élément contenant le nombre de likes
        likes_element = driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[2]/div[2]/div/div/ytd-menu-renderer/div[1]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]')

        # Récupérer le texte du nombre de likes et le convertir en '0' si le texte est "J'aime"
        likes_text = likes_element.text.strip() if likes_element else 'N/A'

        # Retourner 0 si le texte est "J'aime", sinon retourner le texte récupéré
        return 0 if likes_text == "J'aime" else likes_text

    except Exception as e:
        # En cas d'erreur, afficher un message et retourner "N/A"
        print(f"Erreur lors de la récupération du nombre de likes : {str(e)}")
        return 'N/A'


# Récupération de la description vidéo

La fonction `get_description_video(driver, video_url)` permet de récupérer la description d'une vidéo YouTube en utilisant Selenium pour naviguer sur la page de la vidéo.
Elle clique d'abord sur le bouton "Afficher plus" pour développer la description complète, puis extrait le texte de la description en utilisant BeautifulSoup pour parser le contenu HTML.

**Objectif** :
- Accéder à la page d'une vidéo YouTube, afficher la description complète et l'extraire sous forme de texte.

**Contexte** :
- Utile pour la collecte de données sur YouTube afin de récupérer les descriptions de vidéos pour analyse sémantique, évaluation du contenu ou comparaison entre vidéos.
- Pratique pour construire des bases de données de vidéos ou pour extraire des informations contextuelles sur plusieurs vidéos.

**Approche** :
1. Charger la page de la vidéo en utilisant `driver.get(video_url)`.
2. Attendre que l'élément contenant la description soit visible.
3. Tenter de cliquer sur le bouton "Afficher plus" pour afficher la description complète.
4. Extraire le texte de la description à l'aide de `find_element()` et parser le contenu avec BeautifulSoup.
5. Traiter le contenu pour récupérer uniquement le texte pertinent, en ignorant les balises superflues.
6. Retourner la description complète ou "N/A" si l'élément n'est pas trouvé ou qu'une erreur survient.

**Avantages** :
- La fonction permet de récupérer la description complète, y compris les liens et informations supplémentaires masquées par défaut.
- Utilise BeautifulSoup pour extraire et nettoyer le texte, ce qui permet d'obtenir une description bien formatée.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le module `bs4` (BeautifulSoup) doit être installé (`pip install beautifulsoup4`).
- Le pilote de navigateur (ex. `chromedriver`) doit être configuré correctement.
- La page de la vidéo doit être accessible (pas de restrictions d'âge, géographiques, ou de permissions).

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.
- `video_url` : URL de la vidéo YouTube pour laquelle la description doit être extraite.

**Exceptions gérées** :
- `TimeoutException` : Levée si l'élément contenant la description ou le bouton "Afficher plus" n'est pas trouvé dans le délai imparti.
- `NoSuchElementException` : Levée si l'élément est introuvable sur la page.
- `Exception` : Capture toute autre erreur liée au chargement de la page ou à l'extraction de la description.

**Retour** :
- `description` : Chaîne de caractères représentant la description complète de la vidéo, ou "N/A" si la description n'est pas disponible.

In [ ]:

# Définition de la fonction pour extraire la description complète d'une vidéo YouTube
def get_description_video(driver, video_url):
    try:
        # Accéder à l'URL de la vidéo spécifiée par `video_url`
        driver.get(video_url)

        # Attendre que l'élément contenant la description soit présent sur la page
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'yt-attributed-string.style-scope.ytd-text-inline-expander span.yt-core-attributed-string.yt-core-attributed-string--white-space-pre-wrap'))
        )

        # Essayer de cliquer sur le bouton "Afficher plus" pour développer la description complète
        try:
            afficher_plus_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[4]/div[1]/div/ytd-text-inline-expander/tp-yt-paper-button[1]"))
            )
            afficher_plus_button.click()
            print("Le bouton 'Afficher plus' a été cliqué.")
            time.sleep(2)  # Attendre 2 secondes après le clic pour laisser le temps de charger la description complète
        except Exception:
            print("Erreur lors du clic sur 'Afficher plus'.")

        # Localiser l'élément contenant la description complète de la vidéo
        description_video_element = driver.find_element(By.CSS_SELECTOR, 'yt-attributed-string.style-scope.ytd-text-inline-expander span.yt-core-attributed-string.yt-core-attributed-string--white-space-pre-wrap')

        if description_video_element:
            # Extraire le contenu HTML de l'élément description
            html_content = description_video_element.get_attribute('innerHTML')

            # Parser le contenu HTML pour extraire uniquement le texte avec BeautifulSoup
            soup = bs(html_content, 'html.parser')

            # Rechercher tous les éléments <span> présents dans la description
            all_spans = soup.find_all('span')

            # Extraire le texte de chaque <span> tout en filtrant les éléments ayant des liens superflus
            description_texts = []
            for span in all_spans:
                # Vérifier si le <span> a une classe spécifique qui le rend pertinent pour la description
                if 'yt-core-attributed-string--link-inherit-color' in span.get('class', []):
                    description_texts.append(span.get_text().strip())  # Ajouter le texte du <span> dans la liste

            # Combiner tous les morceaux de texte extraits pour former la description finale
            description = ' '.join(description_texts)

            # Retourner la description complète si elle est trouvée, sinon retourner "N/A"
            return description if description else 'N/A'
        else:
            return 'N/A'

    except Exception as e:
        # En cas d'erreur, afficher un message d'erreur et retourner "N/A"
        print(f"Erreur lors de la récupération de la description : {str(e)}")
        return 'N/A'


# Récupération de la date de publication de la vidéo

La fonction `get_date_publication(driver, video_url)` permet de récupérer la date de publication d'une vidéo YouTube en utilisant Selenium pour naviguer sur la page de la vidéo.
Elle localise l'élément HTML contenant la date de publication et retourne le texte correspondant.

**Objectif** :
- Accéder à la page d'une vidéo YouTube et extraire la date de publication affichée.

**Contexte** :
- Utile pour la collecte de données sur YouTube afin de récupérer des informations temporelles sur les vidéos.
- Pratique pour analyser l'évolution des publications ou pour comparer la popularité de vidéos en fonction de leur date de mise en ligne.

**Approche** :
1. Charger la page de la vidéo en utilisant `driver.get(video_url)`.
2. Attendre que l'élément contenant la date de publication soit visible sur la page.
3. Localiser cet élément avec un sélecteur XPath (`By.XPATH`).
4. Extraire le texte représentant la date de publication.
5. Retourner la date de publication ou "N/A" si l'élément n'est pas trouvé ou qu'une erreur survient.

**Avantages** :
- La fonction utilise des techniques de `WebDriverWait` pour s'assurer que l'élément est bien présent avant de tenter de le lire.
- Gère les erreurs et retourne "N/A" si la date de publication n'est pas disponible.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le pilote de navigateur (ex. `chromedriver`) doit être configuré correctement.
- La page de la vidéo doit être accessible (pas de restrictions d'âge, géographiques, ou de permissions).

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.
- `video_url` : URL de la vidéo YouTube pour laquelle la date de publication doit être extraite.

**Exceptions gérées** :
- `TimeoutException` : Levée si l'élément contenant la date de publication n'est pas trouvé dans le délai imparti.
- `NoSuchElementException` : Levée si l'élément est introuvable sur la page.
- `Exception` : Capture toute autre erreur liée au chargement de la page ou à l'extraction de la date.

**Retour** :
- `date_publication` : Chaîne de caractères représentant la date de publication de la vidéo, ou "N/A" si la date n'est pas disponible.


In [ ]:


# Définition de la fonction pour extraire la date de publication d'une vidéo YouTube
def get_date_publication(driver, video_url):
    try:
        # Accéder à l'URL de la vidéo spécifiée par `video_url`
        driver.get(video_url)

        # Attendre que l'élément contenant la date de publication soit présent sur la page
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[4]/div[1]/div/ytd-watch-info-text/div/yt-formatted-string/span[3]'))
        )

        # Localiser l'élément contenant la date de publication de la vidéo
        date_element = driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[4]/div[1]/div/ytd-watch-info-text/div/yt-formatted-string/span[3]')

        # Retourner le texte de la date de publication s'il est trouvé, sinon "N/A"
        return date_element.text.strip() if date_element else 'N/A'

    except Exception as e:
        # En cas d'erreur, afficher un message d'erreur et retourner "N/A"
        print(f"Erreur lors de la récupération de la date de publication : {str(e)}")
        return 'N/A'


# Extraction et organisation de toutes les données

La fonction `get_info(link, driver)` permet d'extraire des informations principales à partir d'une page web en utilisant les bibliothèques `requests` et `BeautifulSoup` pour analyser le contenu HTML.
Si le type de document est une vidéo, elle utilise Selenium pour interagir avec la page YouTube et récupérer des informations supplémentaires telles que le nombre de vues, le nombre de likes, le propriétaire de la vidéo, etc.

**Objectif** :
- Extraire les informations principales d'une ressource (titre, description, type, lieu, source, thématique, etc.).
- Extraire des informations supplémentaires spécifiques aux vidéos si le document est une vidéo.
- Télécharger les fichiers PDF associés à la ressource si présents.

**Contexte** :
- Utile dans le cadre d'un projet de collecte de données pour construire un tableau d'informations à partir de différentes ressources web.
- Conçu pour récupérer à la fois des informations textuelles (titre, description, source) et des métadonnées pour des vidéos YouTube.

**Approche** :
1. Charger la page web en utilisant `requests.get(link)` et créer un objet `BeautifulSoup` pour analyser le contenu.
2. Extraire les informations principales telles que le titre, la description, le type de document, la source, le lieu et la thématique.
3. Vérifier si le document est une vidéo, et dans ce cas utiliser Selenium (`driver`) pour interagir avec la page YouTube correspondante.
4. Récupérer les informations spécifiques à la vidéo (propriétaire, nombre de vues, etc.) si la vidéo est disponible.
5. Télécharger les fichiers PDF associés s'ils sont présents sur la page.
6. Retourner une ligne de données avec toutes les informations extraites.

**Avantages** :
- Combine à la fois des techniques de scraping (BeautifulSoup) et d'automatisation de navigateur (Selenium).
- Gère différents types de contenu (textuel, vidéo, PDF).

**Prérequis** :
- Les modules `requests` et `bs4` doivent être installés (`pip install requests beautifulsoup4`).
- Selenium doit être configuré avec un pilote approprié (ex: `chromedriver`).
- La page de la vidéo doit être accessible (pas de restrictions d'âge, géographiques, ou de permissions).

**Arguments** :
- `link` : URL de la ressource à analyser.
- `driver` : Instance de Selenium WebDriver pour interagir avec les vidéos YouTube.

**Exceptions gérées** :
- `requests.exceptions.RequestException` : Capture les erreurs liées à la requête HTTP.
- `AttributeError` : Capture les erreurs lors de la recherche d'éléments HTML avec BeautifulSoup.
- `Exception` : Capture toute autre erreur liée à l'extraction ou au téléchargement de contenu.

**Retour** :
- `line` : Liste contenant les informations extraites ou `None` en cas d'erreur.

In [ ]:


# Définition de la fonction pour extraire les informations d'une page web
def get_info(link, driver):
    try:
        # Charger la page web spécifiée par `link` et créer un objet `BeautifulSoup` pour analyser le contenu HTML
        page = requests.get(link)
        soup = bs(page.content, 'html.parser')

        # Extraire le titre de la page
        Titre = soup.find('div', id="block-rexbp-page-title")
        Titre = Titre.find('span').string.strip() if Titre else 'N/A'

        # Extraire la description de la ressource
        Description = soup.find('div', class_="field_texte")
        Description_text = []

        # Si une description est trouvée, extraire le texte de chaque paragraphe
        if Description:
            paragraphs = Description.find_all('p')
            if len(paragraphs) > 1:
                for p in paragraphs[:-1]:  # Ignorer le dernier paragraphe
                    Description_text.append(p.get_text().strip())
            elif len(paragraphs) == 1:
                Description_text.append(paragraphs[0].get_text().strip())
            else:
                Description_text = 'N/A'
        else:
            Description_text = 'N/A'
        Description_text = ' '.join(Description_text)  # Combiner tous les paragraphes en une seule chaîne

        # Extraire le type de document
        Type_document = soup.find('div', class_="field_type_document")
        Type_document = Type_document.find('div', class_="name").string if Type_document else 'N/A'

        # Extraire le lieu de la ressource
        territory = soup.find(class_="field_territoire")
        Lieu = territory.text.strip() if territory else 'N/A'

        # Extraire la source de la ressource
        field_texte_div = soup.find('div', class_='field_texte')
        if field_texte_div:
            paragraphs = field_texte_div.find_all('p')
            if paragraphs:
                last_paragraph = paragraphs[-1]
                Source = last_paragraph.get_text().strip()
            else:
                Source = 'N/A'
        else:
            Source = 'N/A'

        # Extraire la thématique de la ressource
        thematique_div = soup.find('div', class_='field_thematique')
        Thematique = thematique_div.find('div', class_='name').string.strip() if thematique_div else 'N/A'

        # Initialiser les informations vidéo
        Propriétaire_video = 'N/A'
        Titre_vidéo = 'N/A'
        Nb_commentaire_video = 'N/A'
        Nb_vues_video = 'N/A'
        Nb_likes_video = 'N/A'
        Description_video = 'N/A'
        Lien_video = 'N/A'
        Date_publication_video = 'N/A'

        # Vérifier si le document est une vidéo et extraire les informations associées
        if Type_document.lower() == 'vidéo':
            video_div = soup.find('div', class_='video-miniature')
            if video_div:
                video_url = video_div.get('data-url')
                if video_url:
                    driver.get(video_url)
                    time.sleep(3)

                    # Refuser les cookies sur YouTube si le pop-up est présent
                    click_refuse_youtube_cookies(driver)

                    if "cette vidéo n'est plus disponible" not in driver.page_source.lower():
                        # Extraire les informations vidéo si la vidéo est disponible
                        Propriétaire_video = get_video_owner(driver, video_url)
                        Titre_vidéo = get_titre_video(driver, video_url)
                        Nb_commentaire_video = get_nombre_commentaire(driver, video_url)
                        Nb_vues_video = get_nombre_vues(driver, video_url)
                        Nb_likes_video = get_nombre_likes(driver, video_url)
                        Description_video = get_description_video(driver, video_url)
                        Date_publication_video = get_date_publication(driver, video_url)
                        Lien_video = video_url
                    else:
                        print("Cette vidéo n'est plus disponible.")

        # Télécharger les fichiers PDF associés s'ils existent
        pdf_div = soup.find('div', class_='field_pdf')
        if pdf_div:
            pdf_links = pdf_div.find_all('a')
            for index, a_tag in enumerate(pdf_links):
                pdf_url = a_tag.get('href')
                pdf_filename = f"{Titre}_pdf_{index + 1}.pdf"
                download_pdf(pdf_url, pdf_filename)

        # Retourner les informations collectées sous forme de liste
        line = [Titre, Thematique, Description_text, Type_document, Source, Lieu, link, Propriétaire_video, Titre_vidéo, Nb_commentaire_video, Nb_vues_video, Nb_likes_video, Description_video, Lien_video, Date_publication_video]
        return line

    except Exception as e:
        # En cas d'erreur, afficher un message et retourner `None`
        print(f"Une erreur est survenue lors de l'extraction des informations : {str(e)}")
        return None


# Pagination et collecte de liens  

La fonction `paginate_and_collect_data(driver, link_list)` permet de naviguer à travers plusieurs pages web et de collecter les liens des articles disponibles.
Elle utilise Selenium pour gérer la pagination et BeautifulSoup pour analyser le contenu de chaque page et extraire les URL d'articles spécifiques.

**Objectif** :
- Parcourir les pages d'un site avec un bouton de pagination "Suivant".
- Collecter les liens des articles pertinents et les ajouter à une liste de liens (`link_list`).

**Contexte** :
- Utilisée pour extraire toutes les ressources (articles, documents) présentes sur un site où le contenu est réparti sur plusieurs pages.
- Conçue pour fonctionner sur des pages avec chargement dynamique, où les articles ne sont visibles qu'après avoir cliqué sur le bouton "Suivant".

**Approche** :
1. Parcourir la page initiale, puis cliquer sur le bouton "Suivant" pour accéder aux pages suivantes.
2. Utiliser `WebDriverWait` pour s'assurer que les articles sont chargés avant d'extraire les informations.
3. Collecter les liens des articles en vérifiant qu'ils contiennent le chemin "/ressource/" et qu'ils ne sont pas déjà présents dans `link_list`.
4. Répéter le processus de pagination jusqu'à ce qu'il n'y ait plus de bouton "Suivant" ou qu'une erreur survienne.
5. Retourner la liste `link_list` contenant tous les liens collectés.

**Avantages** :
- Assure que tous les articles sur plusieurs pages sont correctement collectés.
- Gère les interruptions de pagination et continue jusqu'à ce que toutes les pages aient été traitées.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- BeautifulSoup (`bs4`) doit être installé (`pip install beautifulsoup4`).
- Le pilote Selenium (ex: `chromedriver`) doit être configuré et compatible avec le navigateur utilisé.

**Arguments** :
- `driver` : Instance de Selenium WebDriver contrôlant le navigateur.
- `link_list` : Liste existante (vide ou partiellement remplie) contenant les liens d'articles à collecter.

**Exceptions gérées** :
- `TimeoutException` : Capture les erreurs liées à l'attente d'éléments non présents sur la page.
- `NoSuchElementException` : Capture les erreurs lors de la recherche du bouton de pagination.
- `Exception` : Capture toute autre erreur imprévue liée à la navigation ou à l'extraction des liens.

**Retour** :
- `link_list` : Liste de chaînes de caractères contenant les liens de tous les articles collectés au cours de la pagination.



In [ ]:
# Définition de la fonction pour gérer la pagination et collecter les liens d'articles
def paginate_and_collect_data(driver, link_list):
    # Initialiser le compteur de pages
    page_count = 0

    # Boucle de pagination
    while True:
        print(f"Traitement de la page {page_count + 1}")

        # Attendre que les articles soient chargés sur la page
        WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".views-infinite-scroll-content-wrapper.clearfix .views-row article"))
        )

        # Utiliser BeautifulSoup pour analyser le contenu de la page
        soup = bs(driver.page_source, 'html.parser')
        articles = soup.find_all('article')

        # Collecter les liens des articles en vérifiant leur pertinence
        for article in articles:
            about_attr = article.get('about')
            # Vérifier que le lien contient "/ressource/" et qu'il n'a pas déjà été collecté
            if about_attr and "/ressource/" in about_attr and about_attr not in link_list:
                link_list.append("https://www.dispositif-rexbp.com" + about_attr)  # Ajouter le lien complet

        print(f"Nombre d'éléments dans link_list après la page {page_count + 1} : {len(link_list)}")
        page_count += 1  # Incrémenter le compteur de pages

        # Tenter de trouver le bouton de la page suivante
        try:
            # Localiser le bouton "Suivant" en utilisant un sélecteur CSS
            next_button = driver.find_element(By.CSS_SELECTOR, 'li.pager__item a[rel="next"]')
            if next_button:
                # Cliquer sur le bouton "Suivant" si trouvé
                next_button.click()
                print(f"Cliqué sur le bouton suivant pour la page {page_count + 1}")
                time.sleep(5)  # Pause pour laisser le temps au contenu de charger
            else:
                # Si aucun bouton "Suivant" n'est trouvé, fin de la pagination
                print("Aucun bouton 'Suivant' trouvé, fin de la pagination.")
                break
        except Exception:
            # Gérer l'exception si le bouton "Suivant" est introuvable ou si une autre erreur survient
            print(f"Erreur lors du clic sur le bouton suivant ou fin de la pagination.")
            break

    # Retourner la liste des liens d'articles collectés
    return link_list


# Fonction principale

La fonction `extract_urls_and_info()` est la fonction principale de ce script. Elle initialise le navigateur Web Selenium, recherche les ressources par thématique sur le site, extrait les URLs correspondantes et collecte les informations de chaque ressource trouvée. Les informations collectées sont ensuite exportées vers un fichier Excel.

**Objectif** :
- Effectuer des recherches thématiques sur un site web pour collecter des données.
- Extraire les informations détaillées de chaque ressource disponible (articles, vidéos, etc.).
- Organiser les données collectées et les exporter dans un fichier Excel pour une consultation et une analyse ultérieures.

**Contexte** :
- Conçue pour automatiser le processus de collecte d'informations sur le site `dispositif-rexbp.com`.
- Pratique pour les projets nécessitant un stockage structuré de données provenant de plusieurs pages.

**Approche** :
1. **Initialiser le navigateur Selenium** :
   - Ouvrir une instance de navigateur pour interagir avec le site web et effectuer les recherches.
2. **Définir les thèmes de recherche** :
   - Les thèmes sont définis dans la liste `themes` et chaque thème est traité individuellement.
3. **Recherche et vérification des résultats** :
   - Pour chaque thème, la fonction effectue une recherche et vérifie si des résultats sont trouvés.
   - Si aucun résultat n'est trouvé, passer au thème suivant.
4. **Collecte des URLs et extraction des informations** :
   - Si des résultats sont trouvés, parcourir toutes les pages et extraire les URLs des ressources.
   - Collecter les informations détaillées de chaque URL et les ajouter à la liste `all_data`.
5. **Exporter les données** :
   - Si des données sont collectées, les exporter dans un fichier Excel.
6. **Arrêter le navigateur** :
   - Fermer proprement l'instance de Selenium pour libérer les ressources.

**Avantages** :
- Permet de collecter automatiquement des informations à partir de plusieurs thèmes en une seule exécution.
- Gère les erreurs de recherche, les cas où il n'y a pas de résultats, et les exporte efficacement dans un fichier Excel.

**Prérequis** :
- Le module `selenium` doit être installé (`pip install selenium`).
- Le pilote de navigateur (ex: `chromedriver`) doit être configuré correctement.
- Les fonctions auxiliaires (`search_by_theme`, `check_no_results`, `paginate_and_collect_data`, `get_info`, et `export_to_excel`) doivent être définies.

**Arguments** :
- Aucun argument requis. Cette fonction est autonome et s'exécute directement.

**Exceptions gérées** :
- Les exceptions de Selenium et les erreurs de connexion sont gérées dans les fonctions appelées.

**Retour** :
- Aucun retour, mais les données sont exportées dans un fichier Excel si la collecte est réussie.

In [ ]:
# Fonction principale pour l'extraction des URLs et des informations associées
def extract_urls_and_info():
    # Initialiser le navigateur Chrome
    driver = webdriver.Chrome()

    try:
        # Définir la liste des thèmes à rechercher sur le site
        themes = ["Métiers de l'encadrement", "BIM"]
        all_data = []  # Liste pour stocker toutes les informations collectées

        # Parcourir chaque thème pour effectuer la recherche
        for theme in themes:
            print(f"Recherche pour le thème : {theme}")

            # Accéder à la page de recherche de ressources sur le site
            driver.get("https://www.dispositif-rexbp.com/ressources")

            # Effectuer la recherche par thème en utilisant la fonction `search_by_theme`
            search_by_theme(driver, theme)

            # Vérifier si aucun résultat n'est trouvé pour le thème en cours
            if check_no_results(driver):
                print(f"Pas de résultats pour le thème : {theme}. Passage au thème suivant.")
                continue  # Passer au thème suivant si aucun résultat n'est trouvé

            # Initialiser une liste pour stocker les URLs des ressources collectées
            link_list = []
            link_list = paginate_and_collect_data(driver, link_list)  # Collecter les URLs sur plusieurs pages

            print(f"Nombre total d'éléments dans link_list pour le thème {theme} : {len(link_list)}")

            # Parcourir chaque URL collectée pour extraire les informations détaillées
            for index, url in enumerate(link_list):
                print(f"Traitement de l'URL {index + 1}/{len(link_list)}: {url}")
                
                # Extraire les informations de la ressource à partir de l'URL
                data = get_info(url, driver)
                
                # Si les informations ont été récupérées avec succès, les ajouter à la liste `all_data`
                if data:
                    all_data.append(data)
                    print(f"Données ajoutées avec succès pour l'URL : {url}")
                else:
                    print(f"Échec de la récupération des données pour l'URL : {url}")

        # Afficher le nombre total de ressources collectées
        print(f"Nombre total d'éléments à exporter : {len(all_data)}")

        # Si des données ont été collectées, les exporter dans un fichier Excel
        if all_data:
            export_to_excel(all_data)

    # Assurer la fermeture du navigateur même en cas d'erreur
    finally:
        driver.quit()

# Point d'entrée du script principal
if __name__ == "__main__":
    extract_urls_and_info()
